In [1]:
import asyncio
import random
from ib_insync import IB, Stock, Option, ComboLeg, Contract, LimitOrder, util
import ib_insync
import xlwings as xw
import asyncio 

# Start the event loop for ib_insync. This is important.
ib_insync.util.startLoop()

async def monitor_trade(trade):
    """Monitors the status of a placed trade and prints updates."""
    if not trade:
        print("❌ Invalid trade object passed to monitor.")
        return
    print(f"\n⏳ Monitoring trade {trade.order.orderId} for contract {trade.contract.localSymbol or trade.contract.symbol}...")
    previous_status = None
    while True:
        await ib_insync.asyncio.sleep(1) # Check status every second
        current_status = trade.orderStatus.status

        # Print status only when it changes
        if current_status != previous_status:
            print(f"   Order Status: {current_status}")
            # Optional: Print fill details if available
            if trade.orderStatus.status == 'Filled':
                fill = trade.fills[-1]
                print(f"   Fill Details: {fill.execution.side} {fill.contract.localSymbol} at {fill.execution.price}")
            previous_status = current_status

        # Exit loop if the order is done (Filled, Cancelled, etc.)
        if trade.isDone():
            print(f"✅ Trade finished with final status: {current_status}")
            break

def get_excel_inputs():
    """Reads strategy parameters from the Excel spreadsheet."""
    try:
        book = xw.books.active
        if book is None:
            print("❌ No active Excel workbook found. Please open your spreadsheet.")
            return None
        print(f"ℹ️  Reading from workbook: {book.name}")
        # Use the sheet name from the screenshots
        sht = book.sheets['Option_tws']
    except Exception as e:
        print(f"❌ Error connecting to Excel: {e}")
        print("   Please ensure your spreadsheet is open and the 'Option_tws' sheet exists.")
        return None

    # Define starting row for each strategy based on the provided layout
    strategy_rows = {
        'Call': 6,
        'Put': 16,
        'Synthetic Long': 26,
        'Synthetic Short': 37,
        'Bull Call Spread': 49,
        'Bear Put Spread': 61,
        'Covered Call': 71,
        'Protective Put': 81,
        'Collar': 91,
        'Straddle': 101,
        'Butterfly Spread': 112,
        'Iron Condor': 126,
        'Calendar Spread': 138,
    }

    strategy_name = sht.range('B4').value
    if not strategy_name:
        print("❌ No strategy selected in Excel cell B4.")
        return None

    start_row = strategy_rows.get(strategy_name)
    if start_row is None:
        print(f"❌ Invalid strategy name '{strategy_name}' in Excel cell B4.")
        return None

    params = {'strategy': strategy_name}
    print(f"📖 Reading parameters for '{strategy_name}' starting at row {start_row}...")

    # Read parameters based on the correct cell for each strategy from the screenshots
    if strategy_name == 'Call':
        params['symbol'] = sht.range(f'B{start_row+2}').value
        params['expiry'] = sht.range(f'B{start_row+3}').value
        params['strike'] = sht.range(f'B{start_row+4}').value
        params['action'] = sht.range(f'B{start_row+5}').value
        params['quantity'] = sht.range(f'B{start_row+6}').value
        params['limit_price'] = sht.range(f'B{start_row+7}').value
    elif strategy_name == 'Put':
        params['symbol'] = sht.range(f'B{start_row+1}').value
        params['expiry'] = sht.range(f'B{start_row+2}').value
        params['strike'] = sht.range(f'B{start_row+3}').value
        params['action'] = sht.range(f'B{start_row+4}').value
        params['quantity'] = sht.range(f'B{start_row+5}').value
        params['limit_price'] = sht.range(f'B{start_row+6}').value
    elif strategy_name in ('Synthetic Long', 'Synthetic Short'):
        params['symbol'] = sht.range(f'B{start_row+2}').value
        params['expiry'] = sht.range(f'B{start_row+3}').value
        params['strike'] = sht.range(f'B{start_row+4}').value
        params['quantity'] = sht.range(f'B{start_row+5}').value
        params['limit_price'] = sht.range(f'B{start_row+6}').value
        params['action'] = 'BUY' if strategy_name == 'Synthetic Long' else 'SELL'
    elif strategy_name == 'Bull Call Spread':
        params['symbol'] = sht.range(f'B{start_row+1}').value
        params['expiry'] = sht.range(f'B{start_row+2}').value
        params['lower_strike'] = sht.range(f'B{start_row+3}').value
        params['higher_strike'] = sht.range(f'B{start_row+4}').value
        params['quantity'] = sht.range(f'B{start_row+5}').value
        params['limit_price'] = sht.range(f'B{start_row+6}').value
        params['action'] = 'BUY'  # Debit spread
    elif strategy_name == 'Bear Put Spread':
        params['symbol'] = sht.range(f'B{start_row+0}').value
        params['expiry'] = sht.range(f'B{start_row+1}').value
        params['higher_strike'] = sht.range(f'B{start_row+2}').value
        params['lower_strike'] = sht.range(f'B{start_row+3}').value
        params['quantity'] = sht.range(f'B{start_row+4}').value
        params['limit_price'] = sht.range(f'B{start_row+5}').value
        params['action'] = 'BUY'  # Debit spread
    elif strategy_name == 'Covered Call':
        params['symbol'] = sht.range(f'B{start_row+1}').value
        params['expiry'] = sht.range(f'B{start_row+2}').value
        params['strike'] = sht.range(f'B{start_row+3}').value
        params['quantity'] = sht.range(f'B{start_row+4}').value
        params['limit_price'] = sht.range(f'B{start_row+5}').value
        params['action'] = 'SELL'  # Credit strategy
    elif strategy_name == 'Protective Put':
        params['symbol'] = sht.range(f'B{start_row+1}').value
        params['expiry'] = sht.range(f'B{start_row+2}').value
        params['strike'] = sht.range(f'B{start_row+3}').value
        params['quantity'] = sht.range(f'B{start_row+4}').value
        params['limit_price'] = sht.range(f'B{start_row+5}').value
        params['action'] = 'BUY'  # Debit strategy
    elif strategy_name == 'Collar':
        params['symbol'] = sht.range(f'B{start_row+1}').value
        params['expiry'] = sht.range(f'B{start_row+2}').value
        params['lower_strike'] = sht.range(f'B{start_row+3}').value  # Put strike
        params['higher_strike'] = sht.range(f'B{start_row+4}').value  # Call strike
        params['quantity'] = sht.range(f'B{start_row+5}').value
        params['limit_price'] = sht.range(f'B{start_row+6}').value
        params['action'] = 'BUY' # Typically a net debit or small credit
    elif strategy_name == 'Straddle':
        params['symbol'] = sht.range(f'B{start_row+2}').value
        params['expiry'] = sht.range(f'B{start_row+3}').value
        params['strike'] = sht.range(f'B{start_row+4}').value
        params['quantity'] = sht.range(f'B{start_row+5}').value
        params['limit_price'] = sht.range(f'B{start_row+6}').value
        params['action'] = sht.range(f'B{start_row+7}').value # Read action from Excel
    elif strategy_name == 'Butterfly Spread':
        params['symbol'] = sht.range(f'B{start_row+2}').value
        params['expiry'] = sht.range(f'B{start_row+3}').value
        params['lower_strike'] = sht.range(f'B{start_row+4}').value
        params['strike'] = sht.range(f'B{start_row+5}').value
        params['higher_strike'] = sht.range(f'B{start_row+6}').value
        params['quantity'] = sht.range(f'B{start_row+7}').value
        params['limit_price'] = sht.range(f'B{start_row+8}').value
        params['action'] = 'BUY'  # Debit spread
    elif strategy_name == 'Iron Condor':
        params['symbol'] = sht.range(f'B{start_row+1}').value
        params['expiry'] = sht.range(f'B{start_row+2}').value
        params['lower_put_strike'] = sht.range(f'B{start_row+3}').value
        params['higher_put_strike'] = sht.range(f'B{start_row+4}').value
        params['lower_call_strike'] = sht.range(f'B{start_row+5}').value
        params['higher_call_strike'] = sht.range(f'B{start_row+6}').value
        params['quantity'] = sht.range(f'B{start_row+7}').value
        params['limit_price'] = sht.range(f'B{start_row+8}').value
        params['action'] = 'SELL'  # Credit spread
    elif strategy_name == 'Calendar Spread':
        # Corrected to read from column B based on screenshot
        params['symbol'] = sht.range(f'B{start_row+1}').value
        option_type_val = sht.range(f'B{start_row+2}').value
        params['option_type'] = str(option_type_val).strip().upper()[0]
        params['strike'] = sht.range(f'B{start_row+3}').value
        params['near_expiry'] = str(int(sht.range(f'B{start_row+4}').value))
        params['far_expiry'] = str(int(sht.range(f'B{start_row+5}').value))
        params['quantity'] = sht.range(f'B{start_row+6}').value
        params['limit_price'] = sht.range(f'B{start_row+7}').value
        params['action'] = 'BUY'  # Debit spread

    print(f"✅ Parameters loaded: {params}")
    return params

async def main():
    """Main function to connect, build strategy from Excel, and place the order."""
    params = get_excel_inputs()
    if not params:
        print("❌ Could not retrieve valid inputs from Excel. Exiting.")
        return

    ib = IB()
    try:
        print("Connecting to Interactive Brokers TWS/Gateway...")
        await ib.connectAsync('127.0.0.1', 7497, clientId=random.randint(1, 9999))
        print("✅ Connection successful.")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return

    strategy_name = params['strategy']
    symbol = params.get('symbol')
    if not symbol:
        print(f"❌ Error: 'symbol' is missing or empty in Excel for '{strategy_name}'.")
        await ib.disconnect()
        return

    exchange = 'SMART'
    currency = 'USD'
    contracts_to_qualify = []
    actions = []
    ratios = []

    print(f"\nBuilding strategy: '{strategy_name}' for {symbol}")

    if strategy_name == 'Call':
        leg1 = Option(symbol, params['expiry'], params['strike'], 'C', exchange)
        contracts_to_qualify = [leg1]
        actions = [params['action']]
        ratios = [1]
    elif strategy_name == 'Put':
        leg1 = Option(symbol, params['expiry'], params['strike'], 'P', exchange)
        contracts_to_qualify = [leg1]
        actions = [params['action']]
        ratios = [1]
    elif strategy_name == 'Synthetic Long':
        leg1 = Option(symbol, params['expiry'], params['strike'], 'C', exchange)
        leg2 = Option(symbol, params['expiry'], params['strike'], 'P', exchange)
        contracts_to_qualify = [leg1, leg2]
        actions = ['BUY', 'SELL']
        ratios = [1, 1]
    elif strategy_name == 'Synthetic Short':
        leg1 = Option(symbol, params['expiry'], params['strike'], 'C', exchange)
        leg2 = Option(symbol, params['expiry'], params['strike'], 'P', exchange)
        contracts_to_qualify = [leg1, leg2]
        actions = ['SELL', 'BUY']
        ratios = [1, 1]
    elif strategy_name == 'Bull Call Spread':
        leg1 = Option(symbol, params['expiry'], params['lower_strike'], 'C', exchange)
        leg2 = Option(symbol, params['expiry'], params['higher_strike'], 'C', exchange)
        contracts_to_qualify = [leg1, leg2]
        actions = ['BUY', 'SELL']
        ratios = [1, 1]
    elif strategy_name == 'Bear Put Spread':
        leg1 = Option(symbol, params['expiry'], params['higher_strike'], 'P', exchange)
        leg2 = Option(symbol, params['expiry'], params['lower_strike'], 'P', exchange)
        contracts_to_qualify = [leg1, leg2]
        actions = ['BUY', 'SELL']
        ratios = [1, 1]
    elif strategy_name == 'Covered Call':
        leg1 = Stock(symbol, exchange, currency)
        leg2 = Option(symbol, params['expiry'], params['strike'], 'C', exchange)
        contracts_to_qualify = [leg1, leg2]
        actions = ['BUY', 'SELL']
        ratios = [100, 1]
    elif strategy_name == 'Protective Put':
        leg1 = Stock(symbol, exchange, currency)
        leg2 = Option(symbol, params['expiry'], params['strike'], 'P', exchange)
        contracts_to_qualify = [leg1, leg2]
        actions = ['BUY', 'BUY']
        ratios = [100, 1]
    elif strategy_name == 'Collar':
        leg1 = Stock(symbol, exchange, currency)
        leg2 = Option(symbol, params['expiry'], params['lower_strike'], 'P', exchange)
        leg3 = Option(symbol, params['expiry'], params['higher_strike'], 'C', exchange)
        contracts_to_qualify = [leg1, leg2, leg3]
        actions = ['BUY', 'BUY', 'SELL']
        ratios = [100, 1, 1]
    elif strategy_name == 'Straddle':
        leg1 = Option(symbol, params['expiry'], params['strike'], 'C', exchange)
        leg2 = Option(symbol, params['expiry'], params['strike'], 'P', exchange)
        contracts_to_qualify = [leg1, leg2]
        action = params['action'].upper()
        actions = [action, action]
        ratios = [1, 1]
    elif strategy_name == 'Butterfly Spread':
        leg1 = Option(symbol, params['expiry'], params['lower_strike'], 'C', exchange)
        leg2 = Option(symbol, params['expiry'], params['strike'], 'C', exchange)
        leg3 = Option(symbol, params['expiry'], params['higher_strike'], 'C', exchange)
        contracts_to_qualify = [leg1, leg2, leg3]
        actions = ['BUY', 'SELL', 'BUY']
        ratios = [1, 2, 1]
    elif strategy_name == 'Iron Condor':
        leg1 = Option(symbol, params['expiry'], params['higher_put_strike'], 'P', exchange)
        leg2 = Option(symbol, params['expiry'], params['lower_put_strike'], 'P', exchange)
        leg3 = Option(symbol, params['expiry'], params['lower_call_strike'], 'C', exchange)
        leg4 = Option(symbol, params['expiry'], params['higher_call_strike'], 'C', exchange)
        contracts_to_qualify = [leg1, leg2, leg3, leg4]
        actions = ['SELL', 'BUY', 'SELL', 'BUY']
        ratios = [1, 1, 1, 1]
    elif strategy_name == 'Calendar Spread':
        leg1 = Option(symbol, params['near_expiry'], params['strike'], params['option_type'], exchange)
        leg2 = Option(symbol, params['far_expiry'], params['strike'], params['option_type'], exchange)
        contracts_to_qualify = [leg1, leg2]
        actions = ['SELL', 'BUY']
        ratios = [1, 1]
    else:
        print(f"❌ Strategy '{strategy_name}' is not recognized or implemented.")
        await ib.disconnect()
        return

    if not contracts_to_qualify:
        print("❌ No contracts were defined for the strategy.")
        await ib.disconnect()
        return

    print("Qualifying contracts...")
    qualified_contracts = await ib.qualifyContractsAsync(*contracts_to_qualify)
    if len(qualified_contracts) != len(contracts_to_qualify):
        print("❌ Could not qualify all contracts for the strategy.")
        await ib.disconnect()
        return
    print("✅ All contracts qualified successfully.")

    if len(qualified_contracts) == 1:
        final_contract = qualified_contracts[0]
        order_action = actions[0]
    else:
        combo_legs = [
            ComboLeg(conId=c.conId, ratio=r, action=a, exchange=exchange)
            for c, a, r in zip(qualified_contracts, actions, ratios)
        ]
        final_contract = Contract(
            symbol=symbol, secType='BAG', currency=currency, exchange=exchange, comboLegs=combo_legs
        )
        order_action = params['action']

    order = LimitOrder(
        action=order_action, totalQuantity=params['quantity'], lmtPrice=params['limit_price']
    )

    print(f"\nPlacing order for: {final_contract.localSymbol or final_contract.symbol}")
    print(f"   Action: {order.action}, Quantity: {order.totalQuantity}, Limit Price: {order.lmtPrice}")
    trade = ib.placeOrder(final_contract, order)

    await monitor_trade(trade)

    print("\nDisconnecting from IB.")
    await ib.disconnect()

if __name__ == "__main__":
    asyncio.run(main())

ℹ️  Reading from workbook: Richspread.xlsx
📖 Reading parameters for 'Calendar Spread' starting at row 138...
✅ Parameters loaded: {'strategy': 'Calendar Spread', 'symbol': 'AAPL', 'option_type': 'C', 'strike': 180.0, 'near_expiry': '20250815', 'far_expiry': '20250919', 'quantity': 1.0, 'limit_price': 2.1, 'action': 'BUY'}
Connecting to Interactive Brokers TWS/Gateway...
✅ Connection successful.

Building strategy: 'Calendar Spread' for AAPL
Qualifying contracts...
✅ All contracts qualified successfully.

Placing order for: AAPL
   Action: BUY, Quantity: 1.0, Limit Price: 2.1

⏳ Monitoring trade 5 for contract AAPL...


AttributeError: module 'ib_insync' has no attribute 'asyncio'